In [3]:
import csv
import gym
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import torch
import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
from environment.environment import Environment

from train.train import train_agent

from agents.mab_agent import MAB_Agent
from agents.mc_agent import MC_Agent
from agents.sarsa_agent import SARSA_Agent
from agents.dqn_agent import DQN_Agent

In [5]:
tickers = ["AAPL", "AMZN", "GOOGL", "META", "MSFT", "NVDA", "TSLA"]

data = {i: {t: float(row[t]) for t in tickers} \
    for i, row in enumerate(csv.DictReader( \
    open("data/nasdaq_stock_prices.csv", mode='r'), delimiter=','))
}

In [6]:
INITIAL_BALANCE = 10000
WINDOW_SIZE = 5
EPISODES = 1000
VERBOSE = True
LOW_EPSILON = 0.1
HIGH_EPSILON = 0.5
GAMMA = 0.95
ALPHA = 0.3
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"

environment = Environment(data, window_size=WINDOW_SIZE, initial_balance=INITIAL_BALANCE, verbose=False)

In [7]:
mab_agent_low = MAB_Agent(environment, epsilon=LOW_EPSILON)
mab_agent_high = MAB_Agent(environment, epsilon=HIGH_EPSILON)

mc_agent_low = MC_Agent(environment, epsilon=LOW_EPSILON, gamma=GAMMA)
mc_agent_high = MC_Agent(environment, epsilon=HIGH_EPSILON, gamma=GAMMA)

sarsa_agent_low = SARSA_Agent(environment, epsilon=LOW_EPSILON, gamma=GAMMA)
sarsa_agent_high = SARSA_Agent(environment, epsilon=HIGH_EPSILON, alpha=ALPHA, gamma=GAMMA)

dqn_agent_low = DQN_Agent(environment, epsilon=LOW_EPSILON, gamma=GAMMA, device=DEVICE)
dqn_agent_high = DQN_Agent(environment, epsilon=HIGH_EPSILON, gamma=GAMMA, device=DEVICE)

Device: mps
Device: mps


In [10]:
results_mab_agent_low = train_agent(mab_agent_low, environment, episodes=EPISODES, verbose=VERBOSE)
results_mab_agent_high = train_agent(mab_agent_high, environment, episodes=EPISODES, verbose=VERBOSE)

results_mc_agent_low = train_agent(mc_agent_low, environment, episodes=EPISODES, verbose=VERBOSE)
results_mc_agent_high = train_agent(mc_agent_high, environment, episodes=EPISODES, verbose=VERBOSE)

results_sarsa_agent_low = train_agent(sarsa_agent_low, environment, episodes=EPISODES, verbose=VERBOSE)
results_sarsa_agent_high = train_agent(sarsa_agent_high, environment, episodes=EPISODES, verbose=VERBOSE)

results_dqn_agent_low = train_agent(dqn_agent_low, environment, episodes=EPISODES, verbose=VERBOSE)
results_dqn_agent_high = train_agent(dqn_agent_high, environment, episodes=EPISODES, verbose=VERBOSE)

Training | Épisode 1/1000 — Total reward: 593.35
Training | Épisode 2/1000 — Total reward: -358.83
Training | Épisode 3/1000 — Total reward: 1915.32
Training | Épisode 4/1000 — Total reward: 1943.66
Training | Épisode 5/1000 — Total reward: 1094.71
Training | Épisode 6/1000 — Total reward: -396.98
Training | Épisode 7/1000 — Total reward: 2274.20
Training | Épisode 8/1000 — Total reward: -1031.38
Training | Épisode 9/1000 — Total reward: 2780.23
Training | Épisode 10/1000 — Total reward: -736.95
Training | Épisode 11/1000 — Total reward: 1984.10
Training | Épisode 12/1000 — Total reward: 1461.10
Training | Épisode 13/1000 — Total reward: 56.80
Training | Épisode 14/1000 — Total reward: 385.72
Training | Épisode 15/1000 — Total reward: 212.21
Training | Épisode 16/1000 — Total reward: 5522.55
Training | Épisode 17/1000 — Total reward: 1370.06
Training | Épisode 18/1000 — Total reward: -68.87
Training | Épisode 19/1000 — Total reward: 1710.03
Training | Épisode 20/1000 — Total reward: -3

In [14]:
# 1. Résultats formatés dans un DataFrame
results_df = pd.DataFrame({
    f"MAB {mab_agent_low.epsilon}": results_mab_agent_low,
    f"MAB {mab_agent_high.epsilon}": results_mab_agent_high,
    f"MC {mc_agent_low.epsilon}": results_mc_agent_low,
    f"MC {mc_agent_high.epsilon}": results_mc_agent_high,
    f"SARSA {sarsa_agent_low.epsilon}": results_sarsa_agent_low,
    f"SARSA {sarsa_agent_high.epsilon}": results_sarsa_agent_high,
    f"DQN {dqn_agent_low.epsilon}": results_dqn_agent_low,
    f"DQN {dqn_agent_high.epsilon}": results_dqn_agent_high
})

# 2. Lissage via moyenne glissante
window = 20
ma_df = results_df.rolling(window).mean()
ma_df.columns = [f"{col} (MA)" for col in ma_df.columns]

# 3. Fusion et passage au format long
merged_df = pd.concat([results_df, ma_df], axis=1).reset_index().rename(columns={"index": "Episode"})
long_df = merged_df.melt(id_vars="Episode", var_name="Agent", value_name="Reward")
long_df["Type"] = long_df["Agent"].apply(lambda x: "Moving Average" if "(MA)" in x else "Raw")

# 4. Graphique interactif
fig = px.line(
    long_df[long_df["Type"] == "Moving Average"],
    x="Episode",
    y="Reward",
    color="Agent",
    title="Agent Performance Comparison (Moving Average)",
    log_y=False,
    labels={"Reward": "Total Rewards ($)", "Episode": "Episode"},
    color_discrete_sequence=px.colors.qualitative.Set2
)

# 5. Ajout de phases d'entraînement
n = len(results_df)
fig.add_vrect(x0=0, x1=n//3, fillcolor="blue", opacity=0.05, line_width=0, annotation_text="Early Training", annotation_position="top left")
fig.add_vrect(x0=n//3, x1=2*n//3, fillcolor="green", opacity=0.05, line_width=0, annotation_text="Middle Training", annotation_position="top left")
fig.add_vrect(x0=2*n//3, x1=n, fillcolor="red", opacity=0.05, line_width=0, annotation_text="Late Training", annotation_position="top left")

fig.update_layout(
    font=dict(family="Arial", size=14),
    title_font=dict(size=20),
    xaxis_title="Episode",
    yaxis_title="Total Rewards ($)",
    legend_title_text="Agent",
    hovermode="x unified",
    margin=dict(t=60, l=20, r=20, b=20)
)

fig.show()

# 7. Résumé statistique
print("\n PERFORMANCE SUMMARY")
print(f"{'Agent':<15} {'Mean':>10} {'Std':>10} {'Median':>10} {'Min':>10} {'Max':>10}")
for name, data in zip(results_df.columns, results_df.values.T):
    print(f"{name:<15} {np.mean(data):>10.2f} {np.std(data):>10.2f} {np.median(data):>10.2f} {np.min(data):>10.2f} {np.max(data):>10.2f}")


 PERFORMANCE SUMMARY
Agent                 Mean        Std     Median        Min        Max
MAB 0.1            1162.09    2569.54     733.75   -4303.18   21519.53
MAB 0.5            1176.44    2865.61     888.58   -5849.97   17587.71
MC 0.1            10165.70    8406.56    8013.82   -3920.54   59536.16
MC 0.5             5416.49    4965.06    4391.61   -2987.68   36411.70
SARSA 0.1          1649.77    2712.20    1325.84   -4547.82   19117.46
SARSA 0.5          1283.97    3158.22     963.40   -5853.17   16437.11
DQN 0.05           2700.28    4835.24    1425.85   -5188.81   42255.72


In [15]:
# Basculer l'environnement en mode test
environment.train_mode = False

# Optionnel : désactiver l'exploration pendant le test (si méthode train dispo)
for agent in [mab_agent_low, mab_agent_high, mc_agent_low, mc_agent_high, sarsa_agent_low, sarsa_agent_high]:
    if hasattr(agent, "train"):
        agent.train(False)

# Lancer les tests
test_results_mab_low = train_agent(mab_agent_low, environment, episodes=1, verbose=True)
test_results_mab_high = train_agent(mab_agent_high, environment, episodes=1, verbose=True)

test_results_mc_low = train_agent(mc_agent_low, environment, episodes=1, verbose=True)
test_results_mc_high = train_agent(mc_agent_high, environment, episodes=1, verbose=True)

test_results_sarsa_low = train_agent(sarsa_agent_low, environment, episodes=1, verbose=True)
test_results_sarsa_high = train_agent(sarsa_agent_high, environment, episodes=1, verbose=True)

# Affichage synthétique
print("\n📊 Résultats de test sur les 100 derniers jours :")
print(f"MAB (ε={mab_agent_low.epsilon}):  {test_results_mab_low[0]:.2f}")
print(f"MAB (ε={mab_agent_high.epsilon}): {test_results_mab_high[0]:.2f}")
print(f"MC  (ε={mc_agent_low.epsilon}):   {test_results_mc_low[0]:.2f}")
print(f"MC  (ε={mc_agent_high.epsilon}):  {test_results_mc_high[0]:.2f}")
print(f"SARSA (ε={sarsa_agent_low.epsilon}):  {test_results_sarsa_low[0]:.2f}")
print(f"SARSA (ε={sarsa_agent_high.epsilon}): {test_results_sarsa_high[0]:.2f}")

TypeError: Base_Agent.train() takes 1 positional argument but 2 were given